# Tests for the metrics module

In [ ]:
#| hide 
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from fastcore.test import *
from circadian.metrics import esri
from circadian.lights import LightSchedule

# ESRI

In [ ]:
# esri on darkness should be equal to initial_amplitude
dt = 0.1
time = np.arange(0, 24*7, dt)
light_schedule = np.zeros_like(time)
esri_time, esri_array = esri(time, light_schedule, esri_dt=12.0, initial_amplitude=0.1)
test_eq(np.all(np.isclose(esri_array, 0.1)), True)
# esri of regular schedule low lux
schedule = LightSchedule.Regular(lux=100)
light = schedule(time)
esri_time, esri_array = esri(time, light, esri_dt=12.0)
ground_truth = 0.55 # close to this value
test_eq(np.isclose(np.mean(esri_array), ground_truth, atol=0.01), True)
# esri of regular schedule high lux
schedule = LightSchedule.Regular(lux=10000)
light = schedule(time)
esri_time, esri_array = esri(time, light, esri_dt=12.0)
ground_truth = 0.89 # close to this value
test_eq(np.isclose(np.mean(esri_array), ground_truth, atol=0.01), True)
# input validation
test_fail(lambda: esri(time=1, light_schedule=np.array([1, 2])), contains='time must be a numpy array')
test_fail(lambda: esri(time=np.array([1, 2]), light_schedule=1), contains='light_schedule must be a numpy array')
test_fail(lambda: esri(time=np.array([1, 2]), light_schedule=np.array([1, 2, 3])), contains='time and light_schedule must be the same length')
test_fail(lambda: esri(time=np.array([1, 2, 4]), light_schedule=np.array([1, 2, 3])), contains='time must have a fixed time resolution')
test_fail(lambda: esri(time=np.array([1, 2, 3]), light_schedule=np.array([1, 2, 3]), analysis_days='a'), contains='analysis_days must be an integer')
test_fail(lambda: esri(time=np.array([1, 2, 3]), light_schedule=np.array([1, 2, 3]), analysis_days=0), contains='analysis_days must be greater than 0')
test_fail(lambda: esri(time=np.array([1, 2, 3]), light_schedule=np.array([1, 2, 3]), esri_dt='a'), contains='esri_dt must be a float or an int')
test_fail(lambda: esri(time=np.array([1, 2, 3]), light_schedule=np.array([1, 2, 3]), esri_dt=0), contains='esri_dt must be greater than 0')
test_fail(lambda: esri(time=np.array([1, 2, 3]), light_schedule=np.array([1, 2, 3]), initial_amplitude='a'), contains='initial_amplitude must be a float')
test_fail(lambda: esri(time=np.array([1, 2, 3]), light_schedule=np.array([1, 2, 3]), initial_amplitude=-1), contains='initial_amplitude must be non-negative')

In [ ]:
# test that ESRI warns the users when ESRI is negative
dt = 3.0
time = np.arange(0, 24*7, dt)
schedule = LightSchedule.Regular()
light = schedule(time)
test_warns(lambda: esri(time, light, esri_dt=12.0))